# 1)  получаем lookup от исходного языка

In [ ]:
big_token_dict = {
 "vertolet_token" : "dict.1.1.20191029T212638Z.e3a6524ff15de8ee.53dbf57d908343531709c5f0686045121f0fc6ae",
   "bbk_token" : "dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc", 
"vasya_2019" : "dict.1.1.20191029T212638Z.e3a6524ff15de8ee.53dbf57d908343531709c5f0686045121f0fc6ae",
"vasya_2011" : "dict.1.1.20191105T083743Z.8b9cdebb9755fa57.e42afa31c184e2ced2305b5452809e17bc19c920",
"fanyi0" : "dict.1.1.20191105T095108Z.82f72da803eeedc2.ada6293a0591bed0222d3a8045bd722915f025bb",
"fanyi00" : "dict.1.1.20191105T103102Z.3e0f76aef01d786a.92fea3781473b7c432aa49ee1f3879cb95181a63",
"prvd0" : "dict.1.1.20191105T103348Z.960e0892b02b628b.5746d7d95af92a16d15941b24410707bbb7568c4",
"prvd00" : "dict.1.1.20191105T105231Z.520f147163f275cd.f329913de7b0a719762e89ef84914cbe8c3f3d25",
"prvdfin0": "dict.1.1.20191105T105704Z.a6f94da915862b61.76003fa911b16bbad05aa655d5be9ea1f78b3e1c" 
}

In [ ]:
def get_y_lookup_json(from_lang_words_list, from_lang, to_lang, token_dict, directory_with_handled_files):
    tokens_names_list = []
    tokens_list = []
    for token_name, token in token_dict.items():
        tokens_names_list.append(token_name)
        tokens_list.append(token)
    token_index = 0 
    not_handled_words_list = []
    
    save_folder = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en_2"
   
    handled_words_from_folder = get_handled_words(directory_with_handled_files)
    handled_words_from_folder = set(handled_words_from_folder)
    handled_words_from_folder_2 = get_handled_words(save_folder)
    handled_words_from_folder_2 = set(handled_words_from_folder_2)
    handled_words_from_folder = handled_words_from_folder.union(handled_words_from_folder_2)
    print("handled_all_words", len(handled_words_from_folder))
    
    

    count = 0 
    print("going_to_use token", tokens_names_list[token_index])
    skipped_words_count = 0
    handled_words_count = 0
    for fr_word in tqdm(from_lang_words_list):
        #print(fr_word)
        isnan = False
        #print(count, start_from_count)
        try:
            isnan = math.isnan(fr_word)
        except:
            pass
        if fr_word != "no_equality" and isnan == False and fr_word not in handled_words_from_folder:
            lang_pair = from_lang + "-" + to_lang
            headers = {"key":tokens_list[token_index],
                  "lang":lang_pair,"text":fr_word}
            ddd = requests.get(url, headers).json()
            if 'message' in ddd and ddd['message'] == "Limit of daily requests exceeded":
                token_index += 1
                print("LIMIT REACHED, switch to token",tokens_names_list[token_index] )
                not_handled_words_list.append(fr_word)
            try:
                save_loc = os.path.join(save_folder, fr_word + ".json")
            except:
                print(fr_word, "is ambigious float or smth")
                continue
            try:
                with open(save_loc, 'w') as f:
                    json.dump(ddd, f, indent = 4, ensure_ascii=False)
                    #print("saved at ", save_loc)
            except Exception as E:
                print(fr_word, E)
            #time.sleep(0.0001)
            handled_words_count += 1
        else:
            skipped_words_count += 1
            #print(fr_word, "===>>>>skipped")
            pass
            
        count += 1
        if count %500 == 0:
            handled_words_from_folder = get_handled_words(directory_with_handled_files)
            handled_words_from_folder = set(handled_words_from_folder)
            handled_words_from_folder_2 = get_handled_words(save_folder)
            handled_words_from_folder_2 = set(handled_words_from_folder_2)
            handled_words_from_folder = handled_words_from_folder.union(handled_words_from_folder_2)
            print("handled_words_from_folder", len(handled_words_from_folder))
            print(handled_words_count, "words handled", skipped_words_count, "words skipped\n",)     
    return not_handled_words_list
        

In [ ]:
handled = "/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_fr_en"
#from_ind = 1480+382+133+1858+15
#to_ind = 25000
from_ind = 0
to_ind = len(words_to_be_handled)
not_handled_words = get_y_lookup_json(words_to_be_handled[from_ind:to_ind], "fr", "en", small_token_dict,
                  directory_with_handled_files = han dled)



# 2) объединяем сохраненные файлы в csv


In [ ]:
def parse_from_yandex_lookup(directory, print_output = False):
    word = []
    local_word = []
    pos_list = []
    examples = []
    examples_local = []
    words_from_not_handled_count = 0 
    empty = 0
    for file in tqdm(os.listdir(directory)):
        if print_output == True: print(file)
        current_word = file.split(".")[0]
        
        if "DS_Store" not in file:
            try:
                f = open(os.path.join(directory,file), "r", encoding='utf-8')
            except:
                f = open(os.path.join(directory,file), "r")
            try:
                data = json.load(f, encoding = 'utf-8')
            except:
                data = json.load(f)
            if "def" in data:
                for definition in data['def']:
                    word_current =  definition['text']
                    if 'pos' in definition:
                        global_pos = definition['pos']
                    else:
                        global_pos = 'no_pos_available'
                    for translation in definition['tr']: 
                        if 'pos' in translation:
                            pos = translation['pos']
                        else:
                            pos = global_pos
                        pos_list.append(pos)
                        word.append(word_current)
                        local_word.append(translation['text'])
                        ex_en = []
                        ex_rus = []
                        if 'ex' in translation:
                            for exmpl in translation['ex']:
                                #print(exmpl)
                                ex_en.append(exmpl['text'])
                                ex_rus.append(exmpl['tr'][0]['text'])
                        examples.append(ex_en)
                        examples_local.append(ex_rus)
                if len(data['def']) == 0:
                    empty += 1
            words_from_not_handled_count += 1
    print("empty json", empty)
    print("non_handled_basic_language__words_added_to_dataframe", words_from_not_handled_count)
    data = pd.DataFrame(list(zip(word,local_word,pos_list, examples, examples_local)),columns =['word', 'local_word', 'pos', 'examples','local_examples'])
    return  data


# 3) Запускаем скрипт cross_approach_script к полученному csv. Меняем параметры

# 4) объединяем с прошлым concat_medium_results_and_detect_missing_stuff 